In [85]:
# The data we need
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression



#Other shit

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter,
import matplotlib.ticker as ticker
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

import seaborn as sns
import autograd.numpy as np
from autograd import grad, elementwise_grad
import pandas as pd
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.utils import resample
from sklearn.model_selection import KFold, cross_val_score
plt.rcParams['font.size'] = 14


In [86]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)

    return term1 + term2 + term3 + term4 


# Defining the R2 function, from lecture notes
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)


# Defining the Mean square error, from lecture notes
def MSE(y,ytilde):
    n = len(y)
    return 1/n * np.sum(np.abs(y-ytilde)**2)



# Creating the design matrix, from lecture notes
def create_X(x, y, n ):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2) # Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X


# making the OCS regression
def OLSmethod(X,z):
    return np.linalg.pinv(X.T @ X) @ X.T @ z

#from lecturenotes
def gradientOLS(X,z,beta):
    n=len(z)
    return 2.0/n*X.T @ ((X @ beta)-z)




def StocGradOLS(numberMinibach,numEpochs,X,z,eta):
    betaOLS_stoc = np.random.randn(X.shape[1],1)
    MiniBachSize = int(X.shape[0]/numberMinibach)
    ##### For fixed learning rate coment in the last learningrate
    t_0 = 2 #
    t_1 = 2/(eta) #
    for i in range(numEpochs):
        for j in range(numberMinibach):
            miniBach = np.random.randint(numberMinibach)

            miniBachMin, miniBachMax = MiniBachSize * miniBach,(MiniBachSize) * (miniBach+1)
            t = i*numberMinibach + j #
            learningrate = t_0/(t_1 +t) #
            #learningrate = eta
            betaOLS_stoc -= learningrate *gradientOLS(X_train[miniBachMin: miniBachMax],z_train[miniBachMin:miniBachMax],betaOLS_stoc)
    return betaOLS_stoc


def sigmoid(x):
    return 1/(1+np.exp(-x))
def logistic_reg(x,w):
    return sigmoid(np.dot(x,w))

def logistic_cost(x,w,t,_lambda = False):
    pred = logistic_reg(x,w)
    cost_inner = pred * t + (1 - pred) * (1 - t)
    return -np.sum(np.log(cost_inner)) + _lambda*np.sum(w**2)



def StocGradLogic(numberMinibach, numEpochs, X, z, eta,_lambda):
    # Defining weights
    param = np.random.randn(X.shape[1],1)
    not_nan = False
    
    
    MiniBachSize = int(X.shape[0] / numberMinibach)
    t_0 = 2
    t_1 = 2 / (eta)

    gradient = elementwise_grad(logistic_cost, 1)

    for i in range(numEpochs):
        if not_nan == True:
            break
        for j in range(numberMinibach):
            if np.sum(param) == np.nan:
                print("Too high learningrate")
                not_nan = True
                
            miniBach = np.random.randint(numberMinibach)
            miniBachMin, miniBachMax = MiniBachSize * miniBach, (MiniBachSize) * (miniBach + 1)
            t = i * numberMinibach + j
            #learningrate = t_0 / (t_1 + t)
            learningrate = eta
            #print(X[miniBachMin: miniBachMax], param, z[miniBachMin: miniBachMax],_lambda)
            #print(param)
            param -= learningrate * gradient(X[miniBachMin: miniBachMax], param, z[miniBachMin: miniBachMax],_lambda)
            #print(gradient(X[miniBachMin: miniBachMax], param, z[miniBachMin: miniBachMax],_lambda))
    return param
            
#Ridgeregression
def ridgeregg(X,y, lmb = 0.0001):
    XtX = X.T @ X
    p =np.shape(XtX)[0]
    return np.linalg.pinv(XtX +  lmb * np.identity(p)) @ X.T @ y

#This one have to be checked
def gradientRidge(X,z,beta,lam):
    n=len(z)
    return 2.0/n*X.T @ ((X @ beta)-z) + 2.0/n*lam*beta

def StocGradRidge(numberMinibach,numEpochs,X,z,eta,lam):
    betaRidge_stoc = np.random.randn(X.shape[1],1)
    MiniBachSize = int(X.shape[0]/numberMinibach)
        ##### For fixed learning rate coment in the last learningrate
    t_0 = 2 #
    t_1 = 2/(eta) #
    for i in range(numEpochs):
        for j in range(numberMinibach):
            miniBach = np.random.randint(numberMinibach)

            miniBachMin, miniBachMax = MiniBachSize * miniBach,(MiniBachSize) * (miniBach+1)
            t = i*numberMinibach + j #
            learningrate = t_0/(t_1 +t) #
            #learningrate = eta
            betaRidge_stoc -= learningrate *gradientRidge(X_train[miniBachMin: miniBachMax],z_train[miniBachMin:miniBachMax],betaRidge_stoc,lam)
    return betaRidge_stoc

def bootstrap_ols_stoc(X_train,X_test,z_train,z_test,numberOfStraps,numberMinibach,numEpochs,eta):
    MSEdeglisttest = np.zeros(numberOfStraps)
    MSEdeglisttrain = np.zeros(numberOfStraps)
#### we have decided to not consider the variance and bias 
    #bias = np.zeros(numberOfStraps)
    #variance = np.zeros(numberOfStraps)
    for i in range(numberOfStraps):
        bootX,bootz = resample(X_train,z_train.reshape(-1,1))
        bootbetaOLS = StocGradOLS(numberMinibach,numEpochs,bootX,bootz,eta)
        # Making out model and adding it to a list
        z_pred_test = X_test @ bootbetaOLS
        z_pred_train =  X_train @ bootbetaOLS
        MSEdeglisttest[i] =MSE(z_test,z_pred_test)
        MSEdeglisttrain[i]= MSE(z_train,z_pred_train)
  #      bias[i],variance[i] = biassVariance(z_test,zpred)
    return np.mean(MSEdeglisttest), np.mean(MSEdeglisttrain)#, np.mean(bias), np.mean(variance)

def bootstrap_Ridge_stoc(X_train,X_test,z_train,z_test,numberOfStraps,numberMinibach,numEpochs,eta,lam):
    MSEdeglisttest = np.zeros(numberOfStraps)
    MSEdeglisttrain = np.zeros(numberOfStraps)
#### we have decided to not consider the variance and bias 
    #bias = np.zeros(numberOfStraps)
    #variance = np.zeros(numberOfStraps)
    for i in range(numberOfStraps):
        bootX,bootz = resample(X_train,z_train.reshape(-1,1))
        bootbetaRidge = StocGradRidge(numberMinibach,numEpochs,bootX,bootz,eta,lam)
        # Making out model and adding it to a list
        z_pred_test = X_test @ bootbetaRidge
        z_pred_train =  X_train @ bootbetaRidge
        MSEdeglisttest[i] =MSE(z_test,z_pred_test)
        MSEdeglisttrain[i]= MSE(z_train,z_pred_train)
   #     bias[i],variance[i] = biassVariance(z_test,zpred)
    return np.mean(MSEdeglisttest), np.mean(MSEdeglisttrain)#, np.mean(bias), np.mean(variance)






In [99]:
cancer = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(cancer.data,cancer.target,random_state=0)


#eta = 0.0003
#_lambda = 0.01

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


numberMinibacharray = np.array([4,8,16,32,64])
numEpochsarray = np.array([10,100,1000])
etas = np.logspace(-6,-5,5)
lambdas = np.logspace(0,0,5)


for numEpochs in numEpochsarray:
    #print(numEpochs)
    for numberMinibach in numberMinibacharray:
        
        for i,eta in np.ndenumerate(etas):
            print(eta)
            for j,_lambda in np.ndenumerate(lambdas):
                accuracy_array = np.zeros((len(etas),len(lambdas)))

                weights = StocGradLogic(numberMinibach, numEpochs, X_train_scaled, y_train, eta,_lambda)
                print(weights)
                cancer_train_pred = np.rint(X_train_scaled@weights)
                accuracy_array[i,j] = accuracy_score(cancer_train_pred,y_train)
                
    
    sns.set()

    #### Sets a log scale on the heat plot
    tick = ticker.ScalarFormatter(useOffset=False, useMathText=True)
    tick.set_powerlimits((0,0))

    tx = [u"${}$".format(tick.format_data(x)) for x in lambdas]
    ty = [u"${}$".format(tick.format_data(x)) for x in etas]

    #lambdas_sea = [lambdas[i] for i in range(len(lambdas))]
    #etas_sea =  [etas[i] for i in range(len(etas))]

    fig, ax = plt.subplots(figsize = (10, 10))
    sns.heatmap(data=accuracy_array,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
    ax.set_xlabel(r'$\lambda$')
    ax.set_ylabel(r'$\eta$')
    plt.tight_layout()
    #plt.savefig(f"Ridge_Task_a_adaptive_learn_NumEpochs{numEpochs[epoch_index]}_NumBach{numberMinibach[minibach_index]}_New.pdf")
    plt.show()



"""pred = X_train_scaled@weights
pred = np.rint(pred)
#print(pred)
print(accuracy_score(pred,y_train))"""




1e-06
[[ 0.36869038]
 [-0.45979199]
 [ 0.87529808]
 [-1.26326896]
 [-1.02733208]
 [-0.49350721]
 [ 0.1706958 ]
 [-0.59142846]
 [ 0.85101933]
 [ 1.40220806]
 [ 0.87549243]
 [ 1.08584791]
 [ 0.48410785]
 [-0.85935851]
 [-1.55096578]
 [-0.03914334]
 [ 0.60137796]
 [ 0.81157636]
 [ 0.14578489]
 [-0.99862474]
 [ 0.9895774 ]
 [ 0.29445532]
 [-1.09866043]
 [ 0.69141521]
 [ 0.88086969]
 [ 0.44545787]
 [ 0.46586432]
 [-2.53382781]
 [-0.85906746]
 [-1.0181342 ]]
[[-0.73288574]
 [-0.46713847]
 [-0.98580879]
 [ 1.69656391]
 [ 0.15746217]
 [-0.4919645 ]
 [ 1.19708166]
 [-2.2080058 ]
 [ 0.61121689]
 [-1.31923812]
 [ 0.69063044]
 [ 0.65538168]
 [-0.77461991]
 [ 0.20560275]
 [-0.2589981 ]
 [ 0.72911014]
 [-0.17757713]
 [ 1.07973467]
 [ 0.48423983]
 [ 0.58678771]
 [ 0.69263378]
 [-0.76043732]
 [ 0.38323901]
 [ 0.59070502]
 [ 0.86756005]
 [ 0.77618014]
 [ 0.00301815]
 [-2.42913666]
 [ 0.88830099]
 [-0.96865746]]
[[ 0.78299683]
 [ 0.90669009]
 [-0.1464558 ]
 [ 1.81804761]
 [ 0.12532795]
 [-0.66770651]
 [

[[ 1.45780988]
 [-0.53411415]
 [-0.92840228]
 [ 2.10935299]
 [ 0.97398259]
 [ 0.59677094]
 [ 1.0705872 ]
 [ 0.59702933]
 [-1.95404253]
 [ 0.76524365]
 [-1.43853445]
 [ 0.12898177]
 [-0.10703583]
 [ 1.51167199]
 [ 1.3892841 ]
 [-0.29917187]
 [-0.66256681]
 [-1.16457382]
 [ 0.42776467]
 [-0.51305247]
 [-1.76797842]
 [ 0.07456659]
 [ 0.1131505 ]
 [ 1.37962529]
 [ 0.84680518]
 [ 0.576089  ]
 [ 0.64712429]
 [-1.02109638]
 [-0.02715605]
 [ 0.5808736 ]]
[[ 1.09966026]
 [ 2.58283053]
 [ 1.56660947]
 [-1.13102722]
 [ 1.26950044]
 [-0.23694452]
 [ 0.64690066]
 [-1.36089575]
 [ 1.07403676]
 [-0.43528702]
 [ 0.20881663]
 [-1.4315795 ]
 [ 0.10901291]
 [-0.98296994]
 [ 0.3689049 ]
 [-0.24156553]
 [-1.05069612]
 [-0.19120929]
 [-0.23164422]
 [ 1.45653742]
 [ 0.14035731]
 [ 0.92505612]
 [ 0.18041228]
 [-0.91106514]
 [ 1.66032784]
 [ 0.65844402]
 [-2.90680394]
 [-0.53802977]
 [-1.47233956]
 [-1.5643062 ]]
[[-4.34268545e-01]
 [-2.10742799e-01]
 [ 1.24580232e-01]
 [ 1.93958931e-01]
 [ 7.01048172e-02]
 [-

[[ 0.47506311]
 [-0.45471474]
 [-1.0869896 ]
 [ 0.5463519 ]
 [-1.03233428]
 [ 2.19919868]
 [-0.18586034]
 [ 0.23047506]
 [ 0.13335825]
 [ 0.30482515]
 [ 0.36724409]
 [ 0.31882826]
 [-0.8548479 ]
 [-0.38481987]
 [ 0.54317788]
 [ 0.44121619]
 [-0.97589045]
 [ 0.99261155]
 [-0.24372076]
 [ 0.74914968]
 [ 1.1104156 ]
 [ 0.62066126]
 [-0.25945248]
 [-0.54865862]
 [ 1.33529562]
 [-0.10113475]
 [-1.46275166]
 [-0.80389998]
 [-0.49270912]
 [-0.58534193]]
[[-0.66773722]
 [ 0.2682362 ]
 [-0.23352819]
 [ 0.221342  ]
 [-0.35225418]
 [ 0.08496009]
 [ 0.91884414]
 [ 0.26866947]
 [ 0.94573594]
 [ 0.08963828]
 [-0.07965581]
 [-0.6108393 ]
 [-0.49981453]
 [ 2.81932956]
 [ 0.49327305]
 [-2.10309947]
 [ 0.28461161]
 [ 0.37691087]
 [ 0.6274637 ]
 [-0.29871834]
 [-1.48915171]
 [ 0.15992296]
 [ 0.01661224]
 [ 0.14424705]
 [-0.10358096]
 [ 0.26912816]
 [-0.67554173]
 [ 0.42697544]
 [-0.34565111]
 [-0.21486658]]
[[-0.32175544]
 [ 0.1802925 ]
 [-1.03010795]
 [ 0.40299539]
 [ 0.47255258]
 [ 0.19936553]
 [-0.392

C:\Users\Morty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\autograd\tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
C:\Users\Morty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\autograd\numpy\numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in true_divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
C:\Users\Morty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\autograd\numpy\numpy_vjps.py:34: RuntimeWarning: invalid value encountered in multiply
  defvjp(anp.multiply,    lambda ans, x, y : unbroadcast_f(x, lambda g: y * g),


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
logreg = LogisticRegression()


logreg.fit(X_train, y_train)
print("Train set accuracy from Logistic Regression: {:.2f}".format(logreg.score(X_train,y_train)))
# We scale the data
from sklearn.preprocessing import StandardScaler

# Then perform again a log reg fit
logreg.fit(X_train_scaled, y_train)
print("Train set accuracy scaled data: {:.2f}".format(logreg.score(X_train_scaled,y_train)))
#thereafter we do a PCA with Scikit-learn
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X2D_train = pca.fit_transform(X_train_scaled)
# and finally compute the log reg fit and the score on the training data
logreg.fit(X2D_train,y_train)
print("Train set accuracy scaled and PCA data: {:.2f}".format(logreg.score(X2D_train,y_train)))

